In [1]:
%pip install ultralytics kagglehub wandb albumentations opencv-python

  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached torch-2.8.0-cp313-none-macosx_11_0_arm64.whl.metadata (30 kB)
  Using cached torchvision-0.23.0-cp313-cp313-macosx_11_0_arm64.whl.metadata (6.1 kB)
  Using cached polars-1.33.1-cp39-abi3-macosx_11_0_arm64.whl.metadata (15 kB)
  Using cached ultralytics_thop-2.0.17-py3-none-any.whl.metadata (14 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached charset_normalizer-3.4.3-cp313-cp313-macosx_10_13_universal2.whl.metadata (36 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached urllib3-2.5.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached certifi-2025.8.3-py3-none-any.whl.metadata (2.4 kB)
  Using cached filelock-3.19.1-py3-none-any.whl.metadata (2.1 kB)
  Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached networkx-3.5-py3-none-any.whl.metadata (6.3 kB)
  Using cache

In [ ]:
import kagglehub

path = kagglehub.dataset_download(
    "shlokraval/ppe-dataset-yolov8",
)

print("Path to dataset files:", path)

Resuming download from 2097152 bytes (2523792472 bytes left)...
Resuming download from https://www.kaggle.com/api/v1/datasets/download/shlokraval/ppe-dataset-yolov8?dataset_version_number=1 (2097152/2525889624) bytes left.


100%|██████████| 2.35G/2.35G [16:21<00:00, 2.57MB/s]

Extracting files...


Path to dataset files: /Users/7n100489/.cache/kagglehub/datasets/shlokraval/ppe-dataset-yolov8/versions/1


In [ ]:
import albumentations as A
import cv2

transform = A.Compose(
    [
        A.HorizontalFlip(p=0.5),
        A.Rotate(limit=45, p=0.5),
        A.ShiftScaleRotate(
            shift_limit=(-0.0625, 0.0625),
            scale_limit=(-0.1, 0.1),
            rotate_limit=(-45, 45),
            interpolation=cv2.INTER_LINEAR,
            border_mode=cv2.BORDER_CONSTANT,
            rotate_method="largest_box",
            p=1.0,
        ),  # Random shift, scale, rotate
        A.RandomBrightnessContrast(
            brightness_limit=0.2, contrast_limit=0.2, p=0.5
        ),  # Adjust brightness and contrast
        A.RandomSunFlare(
            flare_roi=(0.1, 0, 0.9, 0.3),
            angle_range=(0.25, 0.75),
            num_flare_circles_range=(5, 15),
            src_radius=200,
            src_color=(255, 200, 100),
            method="physics_based",
            p=1.0,
        ),  
        A.Perspective(scale=(0.05, 0.1), p=0.5),  # Apply perspective transformation
        A.GaussNoise(std_range=(0.1, 0.2), p=0.5),  # Add Gaussian noise
        A.GaussianBlur(
            sigma_limit=(0.1, 2.0), blur_limit=0, p=0.5
        ),  # Apply Gaussian blur
    ],
    bbox_params=A.BboxParams(format="yolo", label_fields=["class_labels"]),
)


/Users/7n100489/Library/CloudStorage/OneDrive-BankofAyudhyaPublicCompanyLimited/Desktop/Workspace/github/ppe-detection/.venv/lib/python3.13/site-packages/albumentations/core/validation.py:114: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)


In [8]:
import os

img_dir = "./ppe-detection-project-dataset-c/versions/1/train/images"
label_dir = "./ppe-detection-project-dataset-c/versions/1/train/labels"
out_img_dir = "./ppe-detection-project-dataset-c/versions/1/train_aug/images"
out_label_dir = "./ppe-detection-project-dataset-c/versions/1/train_aug/labels"

os.makedirs(out_img_dir, exist_ok=True)
os.makedirs(out_label_dir, exist_ok=True)


for img_file in os.listdir(img_dir):
    if not img_file.lower().endswith((".jpg", ".jpeg", ".png")):
        continue

    img_path = os.path.join(img_dir, img_file)
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Load YOLO labels
    label_file = os.path.join(
        label_dir, img_file.replace(".jpg", ".txt").replace(".png", ".txt")
    )
    bboxes = []
    class_labels = []
    if os.path.exists(label_file):
        with open(label_file, "r") as f:
            for line in f.readlines():
                class_id, x, y, w, h = map(float, line.strip().split())
                bboxes.append([x, y, w, h])
                class_labels.append(int(class_id))

    # Apply augmentation
    augmented = transform(image=img, bboxes=bboxes, class_labels=class_labels)
    aug_img = cv2.cvtColor(augmented["image"], cv2.COLOR_RGB2BGR)

    # Save augmented image
    out_img_path = os.path.join(out_img_dir, img_file)
    cv2.imwrite(out_img_path, aug_img)

    # Save augmented labels
    out_label_path = os.path.join(
        out_label_dir, img_file.replace(".jpg", ".txt").replace(".png", ".txt")
    )
    with open(out_label_path, "w") as f:
        for bbox, cid in zip(augmented["bboxes"], augmented["class_labels"]):
            f.write(f"{cid} {' '.join(map(str, bbox))}\n")

In [ ]:
import os

dataset_root = "./ppe-detection-project-dataset-c/versions/1"

dataset_dirs = ["train_aug"]

for split in dataset_dirs:
    img_dir = os.path.join(dataset_root, split, "images")
    lbl_dir = os.path.join(dataset_root, split, "labels")

    if not os.path.exists(img_dir) or not os.path.exists(lbl_dir):
        print(f"Warning: {split} folder not found. Skipping...")
        continue

    img_files = sorted(os.listdir(img_dir))
    lbl_files = sorted(os.listdir(lbl_dir))

    if len(img_files) != len(lbl_files):
        print(f"Warning: Number of images and labels do not match in {split}")
        continue

    for idx, (img_file, lbl_file) in enumerate(zip(img_files, lbl_files), start=1):
        new_name = f"{idx:04d}"
        img_ext = os.path.splitext(img_file)[1]  
        lbl_ext = os.path.splitext(lbl_file)[1]  

        new_img_name = new_name + img_ext
        new_lbl_name = new_name + lbl_ext

        os.rename(os.path.join(img_dir, img_file), os.path.join(img_dir, new_img_name))
        os.rename(os.path.join(lbl_dir, lbl_file), os.path.join(lbl_dir, new_lbl_name))

    print(f"Finished renaming files in {split}")


Finished renaming files in train_aug


In [9]:
import yaml

config = {
    "train": f"{path}/train/images",
    "val": f"{path}/valid/images",
    "test": f"{path}/test/images",
    "nc": 11,
    "names": [
        "Gloves",
        "Goggles",
        "Hardhat",
        "Mask",
        "NO-Gloves",
        "NO-Goggles",
        "NO-Hardhat",
        "NO-Mask",
        "NO-Safety Vest",
        "Person",
        "Safety Vest",
    ],
}

with open("config.yaml", "w") as f:
    yaml.dump(config, f)